# Areas of Interest Data Analysis

In [7]:
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np

from pathlib import Path

import re

import modules.rEYEkerAnalysis as rEYEker

from scipy import stats

from chord import Chord

In [8]:
config_prefix = ['BR', 'BI', 'TR', 'TI']

config_algo_names = ['BinarySearch', 'BubbleSort', 'Factorial', 'Fibonacci', 'IntegerBinary', 'MultiplyMatrix', 'PrimeFactors', 'ReverseString']

config_click_variables = [
    ['A319_01', 'A420_01', 'A220_01', 'A118_01'],
    ['BR18_01', 'BI18_01', 'TR18_01', 'TI18_01'],
    ['BR19_01', 'BI19_01', 'TR19_01', 'TI19_01'],
    ['BR20_01', 'BI20_01', 'TR20_01', 'TI20_01'],
    ['A418_01', 'A317_01', 'A117_01', 'A218_01'],
    ['A219_01', 'A120_01', 'A320_01', 'A417_01'],
    ['A119_01', 'A217_01', 'A419_01', 'A318_01'],
    ['BR21_01', 'BI21_01', 'TR21_01', 'TI21_01']
]

config_answer_variables = [
    ['A310_01', 'A414_01', 'A214_01', 'A106_01'],
    ['BR02_01', 'BI02_01', 'TR02_01', 'TI02_01'],
    ['BR06_01', 'BI06_01', 'TR06_01', 'TI06_01'],
    ['BR10_01', 'BI10_01', 'TR10_01', 'TI10_01'],
    ['A406_01', 'A302_01', 'A102_01', 'A206_01'],
    ['A210_01', 'A114_01', 'A314_01', 'A402_01'],
    ['A110_01', 'A202_01', 'A410_01', 'A306_01'],
    ['BR14_01', 'BI14_01', 'TR14_01', 'TI14_01'],
]

config_time_variables = [
    ['TIME049','TIME070','TIME036','TIME015'],
    ['TIME021','TIME055','TIME005','TIME038'],
    ['TIME006','TIME040','TIME025','TIME061'],
    ['TIME059', 'TIME023', 'TIME042', 'TIME008'],
    ['TIME066','TIME051','TIME017','TIME032'],
    ['TIME034','TIME019','TIME053','TIME064'],
    ['TIME013','TIME030','TIME068','TIME047'],
    ['TIME044','TIME010','TIME057','TIME027'],
]


config_answer_patterns = [
    ['3', '3', '3', '3'],
    ['3.*16.*23.*42.*61.*75.*536','3.*16.*23.*42.*61.*75.*536','3.*16.*23.*42.*61.*75.*536','3.*16.*23.*42.*61.*75.*536'],
    ['6','6','120','120'],
    ['2', '2','2','2'],
    ['1.*0.*0.*0.*1','1.*0.*0.*0.*1','1.*0.*0.*0.*1','1.*0.*0.*0.*1'],
    ['6.*6.*6.*12.*12.*12.*18.*18.*18','6.*6.*6.*12.*12.*12.*18.*18.*18','6.*6.*6.*12.*12.*12.*18.*18.*18','6.*6.*6.*12.*12.*12.*18.*18.*18'],
    ['3.*5','3.*5','3.*5','3.*5'],
    ['gnikcar[tT].*ey[eE]','gnikcar[tT].*ey[eE]','gnikcar[tT].*ey[eE]','gnikcar[tT].*ey[eE]']
]


config_reyeker_settings_path = "data/example.json"

config_datasheet = r'./Book4.xlsx'

config_alpha = 0.05

In [9]:
image_paths = []

for algo_name in config_algo_names:
    image_array = []
    for prefix in config_prefix:
        image_array.append('images/' + prefix + '/' + prefix + '_' + algo_name + '.png')
    image_paths.append(image_array)
    
aoi_datasheets = []
for algo_name in config_algo_names:
    datasheet_array = []
    for prefix in config_prefix:
        datasheet_array.append('data/aoi_categorized/AOI_' + prefix + '_' + algo_name + '.xlsx')
    aoi_datasheets.append(datasheet_array)

In [10]:
df_matrix = []
raw = pd.read_excel(config_datasheet)

# load data
for row, _stuff in enumerate(config_algo_names):
    df_array = []
    for col, _stuff in enumerate(config_prefix):
        df = pd.DataFrame(raw, columns = [config_click_variables[row][col], config_answer_variables[row][col], config_time_variables[row][col]])
        df = df.iloc[1:]
        df = df.dropna()
        df_array.append(df)
    df_matrix.append(df_array)

In [11]:
tmp_df_matrix = []

#filter for right
for row, df_row in enumerate(df_matrix):
    tmp_df_array = []
    for col, df in enumerate(df_row):
        right_answer_pattern = config_answer_patterns[row][col]
        regex = re.compile(right_answer_pattern)
        answer_field = config_answer_variables[row][col]
        
        df_right = pd.DataFrame(columns = [config_click_variables[row][col], config_answer_variables[row][col], config_time_variables[row][col]])
        
        for _idx, df_row in df.iterrows():
            result = regex.match(str(df_row[answer_field]))
            if result is not None:
                df_right = df_right.append(df_row)
                
        tmp_df_array.append(df_right)
    tmp_df_matrix.append(tmp_df_array)
                
df_matrix = tmp_df_matrix

In [12]:
tmp_df_matrix = []

# filter for outliers
for row, df_row in enumerate(df_matrix):
    tmp_df_array = []
    
    for col, df in enumerate(df_row):
        time_field = config_time_variables[row][col]
        data = df[time_field]
        
        cleared_df = None
        if len(data) == 1:
            cleared_dataframe = df
        else:
            cleared_dataframe = df[(np.abs(stats.zscore(data.astype(float))) < 3)]
            
        tmp_df_array.append(cleared_dataframe)
        
    tmp_df_matrix.append(tmp_df_array)
                
df_matrix = tmp_df_matrix

In [13]:
image_matrix = []
for row in range(len(config_algo_names)):
    img_array = []
    for col in range(len(config_prefix)):
        img = rEYEker.load_image(image_paths[row][col])
        img_array.append(img)
    image_matrix.append(img_array)

In [14]:
#cast data to valid format

visual_stimulus_tensor = []

for row in range(len(config_algo_names)):
    visual_stimulus_matrix = []
    
    for col in range(len(config_prefix)):
        visual_stimulus_array = []
        df = df_matrix[row][col]
        variable_name = config_click_variables[row][col]
        
        for _idx, item in df.iterrows():
            data_str = item[variable_name]
            data_str = data_str.strip()
            
            coordinate_str_array = data_str.split(" ")
            coordinates = []
            for coordinate_str in coordinate_str_array:
                coordinate = coordinate_str.split("-")
                coordinate = (int(coordinate[1]))
                coordinates.append(coordinate)
                
            visual_stimulus_array.append(coordinates)
        visual_stimulus_matrix.append(visual_stimulus_array)
    visual_stimulus_tensor.append(visual_stimulus_matrix)

In [19]:
iterative = [0 - 'none', 1 - 'main', 2 - 'Iterative definition', 3 - 'Pre calculation', 4 - 'Iteration Condition', 5 - 'Iteration Step', 6 - 'Return Result']
recursive = [0 - 'none', 1 - 'main', 2 - 'Recursive definition', 3 - 'Pre calculation', 4 - 'Recursive Condition', 5 - 'Recursive Step', 6 - 'Return Result']
order = ['0', '1', '2', '3', '4', '5', '6']

In [16]:
def is_in(df, y):
    for _idx, row in df.iterrows():
        if row["startHeight"] <= y <= row["stopHeight"]:
            return row["Name"]
    return "none"

In [17]:
cols = ["startHeight", "stopHeight", "startWidth", "stopWidth", "Name"]

relation_array = []
for row in range(len(config_prefix)):
    names = None
    if row % 2 == 0:
        names = recursive
    else:
        names = iterative
    
    relation = []
    for i in range(len(names)):
        relation.append([0]*len(names))
    
    for col in range(len(config_algo_names)):
        datasheet = aoi_datasheets[col][row]
        df = pd.read_excel(datasheet)
        
        for data_set in visual_stimulus_tensor[col][row]:
            for idx in range(len(data_set)-1):
                from_field = is_in(df, data_set[idx])
                to_field = is_in(df, data_set[idx+1])
        
                from_field = names.index(from_field)
                to_field = names.index(to_field)
                
                relation[from_field][to_field] = 1 + relation[from_field][to_field] 
                
    relation_array.append(relation)

In [27]:
for matrix in relation_array:
    for idx in range(len(matrix)):
        matrix[idx][idx] = 0

In [28]:
Chord(relation_array[0], order, padding=0.1).to_html("BR_general.html")
Chord(relation_array[1], order, padding=0.1).to_html("BI_general.html")
Chord(relation_array[2], order, padding=0.1).to_html("TR_general.html")
Chord(relation_array[3], order, padding=0.1).to_html("TI_general.html")

In [29]:
from pathlib import Path

file = Path('BR_general.html')
file.write_text(file.read_text().replace('True', 'true'))

file = Path('BI_general.html')
file.write_text(file.read_text().replace('True', 'true'))

file = Path('TR_general.html')
file.write_text(file.read_text().replace('True', 'true'))

file = Path('TI_general.html')
file.write_text(file.read_text().replace('True', 'true'))


559

In [24]:
names = ["Action", "Adventure", "Comedy", "Drama", "Fantasy", "Thriller"]

matrix = [
    [0, 5, 6, 4, 7, 4],
    [5, 0, 5, 4, 6, 5],
    [6, 5, 0, 4, 5, 5],
    [4, 4, 4, 0, 5, 5],
    [7, 6, 5, 5, 0, 4],
    [4, 5, 5, 5, 4, 0]
]

Chord(matrix, names).to_html()